In [1]:
import pandas as pd
import glob as gb
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, CountVectorizer, TfidfTransformer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.pipeline import Pipeline

In [2]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
#from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB 
from sklearn.decomposition import PCA

In [3]:
TrnFlsPth=gb.glob("DTrain/*/*")
TstFlsPth=gb.glob("DTest/*/*")

In [4]:
Tr_lbl=[e.strip().split('\\')[1] for e in TrnFlsPth]
Ts_lbl=[e.strip().split('\\')[1] for e in TstFlsPth]

In [5]:
Y_Trn=pd.get_dummies(Tr_lbl)
Y_Tst=pd.get_dummies(Ts_lbl)

In [6]:
fpt=[open(f) for f in TrnFlsPth]
Trfob=[f.read().replace('\n', ' ') for f in fpt]
all(f.close() for f in fpt)

False

In [7]:
fpt=[open(f) for f in TstFlsPth]
Tsfob=[f.read().replace('\n', ' ') for f in fpt]
all(f.close() for f in fpt)
fpt=None

In [8]:
ps=PorterStemmer()
Trfob_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Trfob]
Tsfob_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Tsfob]

In [9]:
lm=WordNetLemmatizer()
Trfob_lmt=[' '.join([lm.lemmatize(word) for word in text.split(' ')]) for text in Trfob]
Tsfob_lmt=[' '.join([lm.lemmatize(word) for word in text.split(' ')]) for text in Trfob]
Trfob_lmt_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Trfob_lmt]
Tsfob_lmt_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Trfob_lmt]

In [10]:
vc=CountVectorizer(analyzer='word', binary=False, lowercase=True, max_df=10, min_df=1,
                   stop_words='english')
vc_ngm=CountVectorizer(analyzer='word', binary=False, lowercase=True, max_df=10, min_df=1,
                   ngram_range=(1, 3), stop_words='english')
idft=TfidfTransformer()
idft_ngm=TfidfTransformer()

In [11]:
TrDvcX=vc.fit_transform(Trfob_stm)
TrDvc_ngmX=vc_ngm.fit_transform(Trfob_stm)
TrDtvcX=idft.fit_transform(TrDvcX.toarray())
TrDtvc_ngmX=idft_ngm.fit_transform(TrDvc_ngmX.toarray())

In [12]:
TrDvc=pd.DataFrame(TrDvcX.toarray(), columns=vc.get_feature_names())
TrDvc_ngm=pd.DataFrame(TrDvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())
TrDtvc=pd.DataFrame(TrDtvcX.toarray(), columns=vc.get_feature_names())
TrDtvc_ngm=pd.DataFrame(TrDtvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())

In [13]:
print(TrDvc.shape, TrDvc_ngm.shape, TrDtvc.shape, TrDtvc_ngm.shape)

(499, 5175) (499, 89404) (499, 5175) (499, 89404)


In [14]:
TsDvcX=vc.transform(Tsfob_stm)
TsDtvcX=idft.transform(TsDvcX.toarray())
TsDtvc=pd.DataFrame(TsDtvcX.toarray(), columns=vc.get_feature_names())
TsDvc=pd.DataFrame(TsDvcX.toarray(), columns=vc.get_feature_names())

In [15]:
print(TsDvc.shape,TsDtvc.shape)

(185, 5175) (185, 5175)


In [16]:
TsDvc_ngmX=vc_ngm.transform(Tsfob_stm)
TsDvc_ngm=pd.DataFrame(TsDvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())
TsDtvc_ngmX=idft_ngm.transform(TsDvc_ngmX.toarray())
TsDtvc_ngm=pd.DataFrame(TsDtvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())

In [17]:
print(TsDvc_ngm.shape, TsDtvc_ngm.shape)

(185, 89404) (185, 89404)


In [18]:
clf=MultinomialNB()

In [19]:
clf.fit(TrDtvc_ngm, Tr_lbl)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
y=clf.predict(TsDtvc_ngm)

In [21]:
accuracy_score(Ts_lbl, y)

0.3783783783783784

In [22]:
lex=['0','00','000','0000']
exstp=[fv for fv in TrDvc_ngm if(len(fv)<3)]
stpwd=ENGLISH_STOP_WORDS.union(exstp,lex)

In [23]:
clsf=BernoulliNB()

In [24]:
doc_clf = Pipeline([('vect', vc), ('tfidf', idft), ('clf',clsf )])

In [25]:
doc_clf.fit(Trfob, Tr_lbl)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=10, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...se, use_idf=True)), ('clf', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))])

In [26]:
y_pred=doc_clf.predict(Tsfob)

In [27]:
accuracy_score(Ts_lbl, y_pred)

0.2972972972972973

In [28]:
CRT=TrDvc.corr()

In [29]:
cCl=set()
crCl=set()
for cl in CRT:
    lidx=CRT.index[(abs(CRT[cl])>0.7) & (CRT[cl]<1)].tolist()
    for e in lidx:
        cCl.add(e)
        if((e, cl) not in crCl):
            crCl.add((cl, e))            

In [30]:
print(len(crCl), len(cCl))

29612 4415


In [31]:
for (a,b) in crCl:
    if((b, a) in crCl):
        print((a, b), (b, a))
        break

In [32]:
pcmp=PCA()

In [33]:
#pcmp.fit(TrDtvc)
tr=pcmp.fit_transform(TrDtvc)
tr=pd.DataFrame(tr)
for cl in tr:
    mn=sum(tr[cl])/499
    tr[cl]=[e+mn for e in tr[cl].values]
    
tr.shape

(499, 499)

In [34]:
ts=pd.DataFrame(pcmp.transform(TsDtvc))
for cl in ts:
    mn=sum(ts[cl])/185
    for e in ts[cl].values:
        e=e+mn
ts.shape

(185, 499)

In [ ]:
scl=scale(X, axis=0, with_mean=True, with_std=False, copy=True)

In [ ]:
tr=scl.fit_transform(tr)

In [ ]:
clf=MultinomialNB()

In [ ]:
clf.fit(tr, Tr_lbl)

In [ ]:
y=clf.predict(ts)

In [ ]:
accuracy_score(Ts_lbl, y)

In [ ]:
pcmp.get_params()